In [2]:
#model
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#dataprocessing
import pandas as pd
import numpy as np
#from google.colab import files
#uploaded = files.upload()

#data visulization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style


In [5]:
#import io
data_test = pd.read_csv('test.csv')
data_test.info()
data_train = pd.read_csv('train.csv')
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float6

In [6]:
data_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [7]:
data_train.describe()        #to Look out for some empty input data fields

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
total = data_train.isnull().sum().sort_values(ascending = True)
missing_data = pd.concat([total], axis=1, keys=['Total'])
print(missing_data)
#we look at the number of values missing in each of the feature so as to decide which one to drop and which one to complete

             Total
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687


In [9]:
#relation of age and sex with survival
survived = 'survived'
not_survived = 'not survived'
fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(10, 4))
women = data_train[data_train['Sex']=='female']
men = data_train[data_train['Sex']=='male']
ax = sns.distplot(women[women['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[0], kde =False)
ax = sns.distplot(women[women['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[0], kde =False)
ax.legend()
ax.set_title('Female')
ax = sns.distplot(men[men['Survived']==1].Age.dropna(), bins=18, label = survived, ax = axes[1], kde = False)
ax = sns.distplot(men[men['Survived']==0].Age.dropna(), bins=40, label = not_survived, ax = axes[1], kde = False)
ax.legend()
_ = ax.set_title('Male')


In [10]:
#realtin of pclass with survival
sns.barplot(x='Pclass', y='Survived', data=data_train)

In [11]:
#relation of embarked, pclass and sex on survival
FacetGrid = sns.FacetGrid(data_train, row='Embarked', size=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette=None,  order=None, hue_order=None )
FacetGrid.add_legend()

In [12]:
#no need to keep siblings and parents of the pasenger in seperate fields as they can be used to denote whether a passenger was alone or not as a single feature
data = [data_train, data_test]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'not_alone'] = 0
    dataset.loc[dataset['relatives'] == 0, 'not_alone'] = 1
    dataset['not_alone'] = dataset['not_alone'].astype(int)
data_train['not_alone'].value_counts()

1    537
0    354
Name: not_alone, dtype: int64

In [13]:
axes = sns.factorplot('relatives','Survived', data=data_train, aspect = 2.5, )

In [14]:
#no need of PassengerId as feature have always a different value
data_train = data_train.drop(['PassengerId'], axis = 1)


In [15]:
#complete the missing data in age by using mean value and standard deviation
data = [data_train, data_test]

for dataset in data:
    mean = data_train["Age"].mean()
    std = data_test["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    #random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values with random generated values
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = data_train["Age"].astype(int)
data_train["Age"].isnull().sum()

0

In [16]:
#look for maximum value of Embarked as only 2 value to fill
data_train['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [17]:
#complete the embarked data using the mode or most occuring value
c_value = 'S'
data = [data_train, data_test]
for dataset in data:
  dataset['Embarked'] = dataset['Embarked'].fillna(c_value)

In [18]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
Survived     891 non-null int64
Pclass       891 non-null int64
Name         891 non-null object
Sex          891 non-null object
Age          891 non-null int64
SibSp        891 non-null int64
Parch        891 non-null int64
Ticket       891 non-null object
Fare         891 non-null float64
Cabin        204 non-null object
Embarked     891 non-null object
relatives    891 non-null int64
not_alone    891 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 90.6+ KB


In [19]:
#convert features to useful data
#covert name to use titles as a new feature
data = [data_train, data_test]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
#drop the name feature, not necessary
data_train = data_train.drop(['Name'], axis=1)
data_test = data_test.drop(['Name'], axis=1)
data_train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare,relatives,not_alone,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.415264,0.523008,0.381594,32.204208,0.904602,0.602694,1.728395
std,0.486592,0.836071,13.544034,1.102743,0.806057,49.693429,1.613459,0.489615,1.030039
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,21.000000,0.000000,0.000000,7.910400,0.000000,0.000000,1.000000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,1.000000,3.000000,37.000000,1.000000,0.000000,31.000000,1.000000,1.000000,2.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,10.000000,1.000000,5.000000


In [20]:
#convert sex into number
data = [data_train, data_test]#ensure new data is used
gender = {"male": 0, "female": 1}
for dataset in data:
  dataset ['Sex'] = dataset['Sex'].map(gender)
data_train.describe()#ensure changes

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,relatives,not_alone,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.415264,0.523008,0.381594,32.204208,0.904602,0.602694,1.728395
std,0.486592,0.836071,0.477990,13.544034,1.102743,0.806057,49.693429,1.613459,0.489615,1.030039
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,21.000000,0.000000,0.000000,7.910400,0.000000,0.000000,1.000000
50%,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000,37.000000,1.000000,0.000000,31.000000,1.000000,1.000000,2.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,10.000000,1.000000,5.000000


In [21]:
#convert fare from float to int and fill nan with 0
data = [data_train, data_test]
for dataset in data:
  dataset['Fare']= dataset['Fare'].fillna(0)
  dataset['Fare'] = dataset['Fare'].astype(int)
  
data_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,relatives,not_alone,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.415264,0.523008,0.381594,31.785634,0.904602,0.602694,1.728395
std,0.486592,0.836071,0.477990,13.544034,1.102743,0.806057,49.703730,1.613459,0.489615,1.030039
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,21.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000
50%,0.000000,3.000000,0.000000,28.000000,0.000000,0.000000,14.000000,0.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000,37.000000,1.000000,0.000000,31.000000,1.000000,1.000000,2.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.000000,10.000000,1.000000,5.000000


In [22]:
dataset['Ticket'].describe()

count          418
unique         363
top       PC 17608
freq             5
Name: Ticket, dtype: object

In [23]:
#there are a large number of unique tickets so few patterns exist, we drop tickets
data_train = data_train.drop(['Ticket'], axis =1)
data_test = data_test.drop(['Ticket'], axis =1)

In [24]:
#convert embarked to numbers
embark = {"S": 0, "C": 1, "Q": 2}
data = [data_train, data_test] # since data was changed
for dataset in data:
  dataset['Embarked'] = dataset['Embarked'].map(embark)

In [25]:
#working withh age feature
#since age is defined for a large range, we will divide it into categories
data=[data_train, data_test]
for dataset in data:
    dataset['Age'] = dataset['Age'].astype(int)
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

In [26]:
#lets take a look at the changes we made
data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,relatives,not_alone,Title
0,0,3,0,2,1,0,7,NaN,0,1,0,1
1,1,1,1,5,1,0,71,C85,1,1,0,3
2,1,3,1,3,0,0,7,NaN,0,0,1,2
3,1,1,1,5,1,0,53,C123,0,1,0,3
4,0,3,0,5,0,0,8,NaN,0,0,1,1


In [27]:
#getting rid of the cabi feature
data_train = data_train.drop(['Cabin'], axis = 1)
data_test = data_test.drop(['Cabin'], axis = 1)

In [28]:
data_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,3.466891,0.523008,0.381594,31.785634,0.361392,0.904602,0.602694,1.728395
std,0.486592,0.836071,0.477990,1.878851,1.102743,0.806057,49.703730,0.635673,1.613459,0.489615,1.030039
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,14.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000,5.000000,1.000000,0.000000,31.000000,1.000000,1.000000,1.000000,2.000000
max,1.000000,3.000000,1.000000,6.000000,8.000000,6.000000,512.000000,2.000000,10.000000,1.000000,5.000000


In [29]:
#categorizing the fare feature
data = [data_train, data_test]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5
    dataset['Fare'] = dataset['Fare'].astype(int)

In [30]:
data_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Title
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,3.466891,0.523008,0.381594,1.523008,0.361392,0.904602,0.602694,1.728395
std,0.486592,0.836071,0.477990,1.878851,1.102743,0.806057,1.250743,0.635673,1.613459,0.489615,1.030039
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000
75%,1.000000,3.000000,1.000000,5.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000
max,1.000000,3.000000,1.000000,6.000000,8.000000,6.000000,5.000000,2.000000,10.000000,1.000000,5.000000


In [31]:
#create new feature as age and class are related(saw that earlier)  age*class
data = [data_train, data_test]
for dataset in data:
  dataset['Age Class']= dataset['Age']*dataset['Pclass']

In [32]:
data_train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Title,Age Class
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,3.466891,0.523008,0.381594,1.523008,0.361392,0.904602,0.602694,1.728395,7.557800
std,0.486592,0.836071,0.477990,1.878851,1.102743,0.806057,1.250743,0.635673,1.613459,0.489615,1.030039,4.867892
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000
50%,0.000000,3.000000,0.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,6.000000
75%,1.000000,3.000000,1.000000,5.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,12.000000
max,1.000000,3.000000,1.000000,6.000000,8.000000,6.000000,5.000000,2.000000,10.000000,1.000000,5.000000,18.000000


In [33]:
#creating another feature fare per person (calling it fpp)
data = [data_train, data_test]
for dataset in data:
  dataset['fpp'] = dataset['Fare']/(dataset['relatives']+1)
  dataset['fpp'] = dataset['fpp'].astype(int)
data_train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Title,Age Class,fpp
0,0,3,0,2,1,0,0,0,1,0,1,6,0
1,1,1,1,5,1,0,3,1,1,0,3,5,1
2,1,3,1,3,0,0,0,0,0,1,2,9,0
3,1,1,1,5,1,0,3,0,1,0,3,5,1
4,0,3,0,5,0,0,1,0,0,1,1,15,1
5,0,3,0,2,0,0,1,2,0,1,1,6,1
6,0,1,0,6,0,0,3,0,0,1,1,6,3
7,0,3,0,0,3,1,2,0,4,0,4,0,0
8,1,3,1,3,0,2,1,0,2,0,3,9,0
9,1,2,1,1,1,0,2,1,1,0,3,2,1


In [34]:
#start with the model
X_train = data_train.drop("Survived", axis = 1)
train_targets = data_train["Survived"]
X_test = data_test.drop("PassengerId", axis = 1).copy()
X_test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,relatives,not_alone,Title,Age Class,fpp
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.363636,3.413876,0.447368,0.392344,1.528708,0.464115,0.839713,0.605263,1.741627,7.825359,0.813397
std,0.841838,0.481622,1.880991,0.896760,0.981429,1.310496,0.685516,1.519072,0.489380,1.006091,5.401894,1.007698
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000
50%,3.000000,0.000000,4.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,6.000000,1.000000
75%,3.000000,1.000000,5.000000,1.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,12.000000,1.000000
max,3.000000,1.000000,6.000000,8.000000,9.000000,5.000000,2.000000,10.000000,1.000000,5.000000,18.000000,5.000000


In [36]:
#I'm using the random forest calssifier fro this model after doing some research on this one
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, train_targets)

Y_prediction = random_forest.predict(X_test)
#print(Y_prediction)
#print(np.prod(Y_prediction.shape))
random_forest.score(X_train, train_targets)
acc_random_forest = round(random_forest.score(X_train, train_targets) * 100, 2)
y=pd.Series(Y_prediction)
pid = data_test["PassengerId"]
pid = pd.Series(pid)
df = pd.DataFrame({"PassengerId": pid, "Survived": y})
#print (df)
#print(data_test["PassengerId"])

In [37]:
print(acc_random_forest)
df.to_csv('submission.csv', index= False)
#The model is 91.92% accurate

91.02
